In [24]:
import pandas as pd
import numpy as np

def append_audio_to_annotated(annot_path, audio_path, out_path=None, fs=256):
    """
    Appends audio envelope columns to the existing merged_annotated dataframe.
    Keeps all original columns, trims audio to match respiration length.
    """
    # --- Load annotated dataframe (full physio/eeg merge) ---
    df_annot = pd.read_csv(annot_path)

    # Add time column if missing
    if "time_s" not in df_annot.columns:
        df_annot["time_s"] = np.arange(len(df_annot)) / fs

    # --- Load audio envelope dataframe ---
    df_audio = pd.read_csv(audio_path)

    # Trim audio to length of annotated data
    t_max = df_annot["time_s"].iloc[-1]
    df_audio_trim = df_audio[df_audio["time_s"] <= t_max].copy()

    # --- Merge by nearest time ---
    df_out = pd.merge_asof(
        df_annot.sort_values("time_s"),
        df_audio_trim.sort_values("time_s"),
        on="time_s",
        direction="nearest"
    )

    # Save if requested
    if out_path:
        df_out.to_csv(out_path, index=False)
        print(f"[OK] Saved annotated + audio → {out_path}")

    return df_out


In [27]:
annot_path = "./data/processed/sub-02/tables/merged_annotated_cut.csv"
audio_path = "./data/processed/sub-02/audio/sea_envelopes_curves.csv"
out_path   = "./data/processed/sub-02/tables/merged_annotated_with_audio.csv"

df_full = append_audio_to_annotated(annot_path, audio_path, out_path=out_path, fs=256)

print(df_full.columns)
print(df_full.head())


C:\Users\andre\AppData\Local\Temp\ipykernel_37908\3792783424.py:10: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_annot = pd.read_csv(annot_path)


[OK] Saved annotated + audio → ./data/processed/sub-02/tables/merged_annotated_with_audio.csv
Index(['ecg', 'respiration', 'physio_triggers', 'condition_triggers',
       'sequence', 'battery', 'flags', 'EEG-ch1', 'EEG-ch2', 'EEG-ch3',
       'EEG-ch4', 'EEG-ch5', 'EEG-ch6', 'EEG-ch7', 'EEG-ch8', 'EEG-ch9',
       'EEG-ch10', 'EEG-ch11', 'EEG-ch12', 'EEG-ch13', 'EEG-ch14', 'EEG-ch15',
       'EEG-ch16', 'EEG-ch17', 'EEG-ch18', 'EEG-ch19', 'EEG-ch20', 'EEG-ch21',
       'EEG-ch22', 'EEG-ch23', 'EEG-ch24', 'EEG-ch25', 'EEG-ch26', 'EEG-ch27',
       'EEG-ch28', 'EEG-ch29', 'EEG-ch30', 'EEG-ch31', 'EEG-ch32',
       'eeg_triggers', 'condition_names', 'time_s', 'env_broad',
       'env_swell_0p3hz', 'env_swell_0p1hz', 'env_mid_1_5hz'],
      dtype='object')
           ecg   respiration  physio_triggers  condition_triggers  sequence  \
0 -2752.846564  52500.224304              0.0         2072.413184     18144   
1 -2856.556719  52066.183090              0.0         1965.702278     18144   
